In [1]:
import requests
from bs4 import BeautifulSoup
import csv

class Scraper ():

    
    #크롤링할 url을 초기값으로 설정
    def __init__(self):
        self.url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date="
        
        
    #url에 연결해서, beautifulsoup을 이용해서 정렬된 soup데이터로 리턴.
    def get_html(self, date):
        res = requests.get(self.url+str(date))

        if res.status_code != 200:
            print("Request Error : ", res.status_code)
    
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        return soup

 
    #등수 변동이 na, up, down으로 제공되기 때문에, = + -로 변경합니다.
    def change_direction(self, L):
        if L == 'na':
            L = '='
        elif L == 'up':
            L = '+'
        else:
            L = '-'

        return L

 
    #순위 데이터를 가져와서, 이름, 변동여부, 변동등수를 가져와서 csv파일에 저장합니다.
    def get_data(self, soup, date):
        #순위 데이터를 가져오자
        listRanking = soup.find(class_ = "list_ranking")
       
        movieName = []
        movieUpdown = []
        movieNum = []

        #tr태그를 찾아서 각 tr태그에서 tit3, arrow, range ac를 가져옵니다
        for tr_tag in listRanking.find_all('tr'):
            tag = tr_tag.find(class_='tit3')
            if tag:
                movieName.append(tag.text.replace("\n",""))

            updown = tr_tag.find(class_='arrow')
            if updown:
                movieUpdown.append(self.change_direction(updown["alt"]))

            num = tr_tag.find(class_='range ac')
            if num:
                movieNum.append(num.text)

            
        self.write_csv(date, movieName, movieUpdown, movieNum)
        
        
    #CSV파일로 저장합니다. UTF-8을 사용하면, 한글이 깨지기때문에, euc-kr형식으로 저장합니다.
    def write_csv(self, date, movieName, movieUpdown, movieNum):
        #파일은 데이터 추가, newline 시 공백 추가
        file = open("movieRank.csv", "a", newline="", encoding  = 'euc-kr')
        
        wr = csv.writer(file)
        for i in range(len(movieName)):
            wr.writerow([str(date), str(i + 1), movieName[i], movieUpdown[i], movieNum[i]])
            
        file.close()
        
    #movieRank라는 csv파일을 생성하고, 날짜마다 한번씩 크롤링해옵니다.
    def scrap(self, startdate, finishdate):
        
        #처음에 실행할 때는 w로 실행하여, 기존의 데이터가 있으면 초기화
        file = open("movieRank.csv", "w", newline = "", encoding  = 'euc-kr')
        wr = csv.writer(file)
        wr.writerow(["Date","Rank.", "Name", "UpDown", "Change_Amount"])
        file.close()
        
        for i in range(finishdate - startdate +1):
            soup = self.get_html(startdate + i)
            self.get_data(soup, startdate + i)

In [2]:
s = Scraper()
s.scrap(20200413,20200414)